## Appendix Source Code for FAHP

In [ ]:
import numpy as np
import pandas as pd

A1 = [0, 0, 0, 0, 0, 0]  # 记录模糊互补矩阵每行和/Record the sum of each row of the fuzzy complementary matrix
R1 = [0, 0, 0, 0, 0, 0]  # 记录一致性矩阵每行积/Record the product of each row of the consistency matrix
W = [0, 0, 0, 0, 0, 0]
R = np.zeros((6, 6), dtype=np.float)
SUM = 0
global N
N = 6

# 模糊互补矩阵/Fuzzy Complementary Matrix
A = np.array([
    [0.50, 0.75, 0.80, 0.60, 0.50, 0.55],
    [0.25, 0.50, 0.65, 0.50, 0.30, 0.40],
    [0.20, 0.35, 0.50, 0.40, 0.30, 0.40],
    [0.40, 0.50, 0.60, 0.50, 0.30, 0.40],
    [0.50, 0.70, 0.70, 0.70, 0.50, 0.75],
    [0.45, 0.60, 0.60, 0.60, 0.25, 0.50]
])

for i in range(N):
    A1[i] = 0  # 记录每行的和/Record the sum of each row
    for j in range(N):
        A1[i] += A[i][j]
print(A1)

# 转换成模糊一致性矩阵/Convert to Fuzzy Consistency Matrix
for i in range(N):
    for j in range(N):
        R[i][j] = (A1[i] - A1[j]) / (2 * N) + 0.5
        
# 幂积法求单层权重/Power product method to find single layer weights
for i in range(N):
    R1[i] = 1
    for j in range(N):
        R1[i] *= R[i][j]
    W[i] = pow(R1[i], 0.2)
    SUM += W[i]
    
for i in range(N):
    W[i] = W[i] / SUM
print(W)

## Appendix Source Code for TOPSIS

In [ ]:
def topsis(data, weight=None):
    data = data / np.sqrt((data ** 2).sum())  # normalized

    Z = pd.DataFrame([data.min(), data.max()], index=['负理想解/negative ideal solution', '正理想解/positive ideal solution'])  # best and worst solution

    weight = entropy_weight(data) if weight is None else np.array(weight)  # distance
    Result = data.copy()
    Result['正理想解'] = np.sqrt(((data - Z.loc['正理想解']) ** 2 * weight).sum(axis=1))
    Result['负理想解'] = np.sqrt(((data - Z.loc['负理想解']) ** 2 * weight).sum(axis=1))

    # composite score index
    Result['综合得分指数/composite score index'] = Result['负理想解'] / (Result['负理想解'] + Result['正理想解'])
    Result['sequence'] = Result.rank(ascending=False)['综合得分指数']

    return Result, Z, weight